In [ ]:
!pip install git+https://github.com/rwightman/pytorch-image-models
import timm

In [ ]:
import sys
import os
import math
import time
import random
import gc
from collections import defaultdict

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

In [ ]:
class CFG:
    seed = 46
    debug = False
    n_fold = 4
    n_epoch = 11
    height = 480
    width = 480
    model_name = "efficientnet_b0"
    lr = 1e-4
    min_lr = 1e-06
    weight_decay = 1e-4
    T_max = 10
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Using device {CFG.device}")

def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(CFG.seed)

In [ ]:
av = pd.read_csv("../input/seti-adversarial-validation/oof.csv")
tmp = pd.read_csv("../input/seti-breakthrough-listen/train_labels.csv")
tmp["label"] = tmp["target"]
av = pd.merge(av, tmp[["id", "label"]], on="id", how="left")
av.query("target == 0 and pred > 1e-4")

In [ ]:
train = av.query("target == 0 and pred > 1e-4").reset_index(drop=True)
train["target"] = train["label"]
train = train.drop(["label", "pred"], axis=1)
test = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
train['file_path'] = train['id'].apply(lambda x:
                                      f"../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy")
test['file_path'] = test['id'].apply(lambda x:
                                    f"../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy")

print(train['target'].value_counts())

if CFG.debug:
    CFG.n_epoch = 2
    train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)
    test = test.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)

skf = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for fold, (_, val_index) in enumerate(skf.split(train, train["target"])):
    train.loc[val_index, 'fold'] = fold
train['fold'] = train['fold'].astype(int)
train.groupby(['fold', 'target']).size()

In [ ]:
def spec_augment(x, alpha=0.1):
    t0 = torch.randint(0, x.shape[0], size=(1,))[0]
    delta = torch.randint(0, int(x.shape[0]*alpha), size=(1,))[0]
    x[t0:min(t0+delta, x.shape[0])] = 0
    t0 = torch.randint(0, x.shape[1], size=(1,))[0]
    delta = torch.randint(0, int(x.shape[1]*alpha), size=(1,))[0]
    x[:, t0:min(t0+delta, x.shape[1])] = 0
    return x

class SpecAugment(ImageOnlyTransform):
    def apply(self, img, **params):
        return spec_augment(img)

DA_DICT = {
    "vflip": A.VerticalFlip(p=0.5),
    "shift_scale_rotate": A.ShiftScaleRotate(rotate_limit=0, p=0.5),
    "motion_blur": A.MotionBlur(p=0.5),
}

In [ ]:
def get_transforms(da: bool):
    if da:
        return A.Compose([
            A.Resize(CFG.height, CFG.width),
            A.OneOf(list(DA_DICT.values())),
            ToTensorV2(),
        ])

    else:
        return A.Compose([
            A.Resize(CFG.height, CFG.width),
            ToTensorV2(),
        ])

    
class SETIDataset(Dataset):
    def __init__(self, df, transform=None, label=True):
        self.df = df
        self.file_names = df['file_path'].values
        if label:
            self.labels = df["target"].values
        else:
            self.labels= None
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = np.load(self.file_names[idx]).astype(np.float32) # (6, 273, 256)
        image = np.vstack(image).transpose((1, 0)) # (256, 1638)
        if self.transform:
            image = self.transform(image=image)['image']
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
        if self.labels is not None:
            label = torch.tensor(self.labels[idx]).float()
            return image, label
        else:
            return image
    
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = torch.distributions.beta.Beta(alpha, alpha).sample()
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(CFG.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [ ]:
class SETIModel(nn.Module):
    def __init__(self, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(CFG.model_name, pretrained=pretrained, in_chans=1)
        self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, 1)

    def forward(self, x):
        return self.backbone(x)

In [ ]:
def criterion(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs, targets)

def train_one_epoch(model, optimizer, dataloader, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:         
        images = images.to(CFG.device)
        labels = labels.to(CFG.device)
        batch_size = images.size(0)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()
                
        running_loss += loss.item() * batch_size
        dataset_size += batch_size 
        epoch_loss = running_loss/dataset_size
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

@torch.no_grad()
def valid_one_epoch(model, dataloader, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    TARGETS = []
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, (images, labels) in bar:        
        images = images.to(CFG.device)
        labels = labels.to(CFG.device)
        batch_size = images.size(0)
        
        outputs = model(images)
        loss = criterion(outputs.view(-1), labels)
        
        running_loss += loss.item() * batch_size
        dataset_size += batch_size
        
        epoch_loss = running_loss/dataset_size
        
        PREDS.append(outputs.sigmoid().cpu().detach().numpy())
        TARGETS.append(labels.view(-1).cpu().detach().numpy())
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    TARGETS = np.concatenate(TARGETS)
    PREDS = np.concatenate(PREDS)
    val_auc = roc_auc_score(TARGETS, PREDS)
    gc.collect()
    
    return epoch_loss, val_auc, PREDS

In [ ]:
def run(model, optimizer, scheduler, train_loader, valid_loader, fold):    
    best_epoch_auc = 0
    best_preds = None
    history = defaultdict(list)
    
    for epoch in range(1, CFG.n_epoch + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, train_loader, epoch)
        valid_epoch_loss, valid_epoch_auc, preds = valid_one_epoch(model, valid_loader, epoch)
        scheduler.step()
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(valid_epoch_loss)
        history['Valid AUC'].append(valid_epoch_auc)
        
        print(f'Valid AUC: {valid_epoch_auc}')
        
        if valid_epoch_auc >= best_epoch_auc:
            print(f"Validation AUC Improved ({best_epoch_auc} ---> {valid_epoch_auc})")
            best_epoch_auc = valid_epoch_auc
            best_preds = preds
            torch.save(model.state_dict(), f"{CFG.model_name}_{fold}.pth")
            print("Model Saved")
            
        print()
        torch.cuda.empty_cache()
        gc.collect()

    print(f"Best AUC for Fold {fold}: {best_epoch_auc:.4f}")
    
    return best_preds, history

def prepare_data(fold):
    df_train = train[train.fold != fold].reset_index(drop=True)
    df_valid = train[train.fold == fold].reset_index(drop=True)
    
    train_dataset = SETIDataset(df_train, transform=get_transforms(True))
    valid_dataset = SETIDataset(df_valid, transform=get_transforms(False))

    train_loader = DataLoader(train_dataset, batch_size=32, 
                              num_workers=4, shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=128, 
                              num_workers=4, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
train["pred"] = 0
scores = []
histories = []
for fold in range(CFG.n_fold):
    print(f"===== fold {fold} ======")
    model = SETIModel()
    model.to(CFG.device)
    train_loader, valid_loader = prepare_data(fold=fold)
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr, last_epoch=-1
    )
    preds, history = run(model, optimizer, scheduler, train_loader, valid_loader, fold)
    train.loc[train.fold == fold, "pred"] = preds
    histories.append(history)
    scores.append(np.max(history['Valid AUC']))
    
    torch.cuda.empty_cache()
    gc.collect()

print("===== Result Summary =====")
print(f"Average AUC: {np.mean(scores):.5f} | Std: {np.std(scores):.5f}")

In [ ]:
print("===== Result Summary =====")
print(f"Average AUC: {np.mean(scores):.5f} | Std: {np.std(scores):.5f}")

In [ ]:
print(f"OOF AUC: {roc_auc_score(train.target, train.pred):.6f}")
display(train)
train[["id", "target", "pred"]].to_csv("oof.csv", index=False)

In [ ]:
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]
fig = plt.figure()
for i in range(CFG.n_fold):
    plt.plot(histories[i]['Train Loss'], label=f'Train {i}', c=colors[i], linestyle="-", marker=".")
    plt.plot(histories[i]['Valid Loss'], label=f'Valid {i}', c=colors[i], linestyle="--", marker=".")
    
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss Curve');

In [ ]:
fig = plt.figure()
for i in range(CFG.n_fold):
    plt.plot(histories[i]['Valid AUC'], label=str(i), marker=".")
plt.legend()
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.title('AUC Curve');

# Inference

In [ ]:
@torch.no_grad()
def predict(model, dataloader):
    model.eval()
    preds = [] 
    index = 0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, images in bar:
        images = images.to(CFG.device)
        pred = model(images).sigmoid().to('cpu').numpy().ravel()
        preds += pred.tolist()
    return np.array(preds)

torch.cuda.empty_cache()
gc.collect()

test_dataset = SETIDataset(test, transform=get_transforms(False), label=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, 
                         num_workers=4, pin_memory=True)
all_predictions = np.zeros((CFG.n_fold, len(test_dataset)))

for fold in range(CFG.n_fold):
    model = SETIModel(pretrained=False)
    model.load_state_dict(torch.load(f"{CFG.model_name}_{fold}.pth"))
    model.to(CFG.device)
    
    all_predictions[fold] = predict(model, test_loader)
    del model
    torch.cuda.empty_cache()
    gc.collect()
    

test["target"] = all_predictions.mean(axis=0)
print(all_predictions)
display(test[['id', 'target']])
test[['id', 'target']].to_csv('submission.csv', index=False)

